In [1]:
"""

To recreate the same results with the pseudo-random numbers simply run all the cells once in order from top to bottom

"""


from utils.histogram import Histo1D
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

seed = 7654
rng = np.random.default_rng(seed)

# defining all the constants

pi = np.pi

qed_coupling = 1/129

qcd_coupling = 0.118

zboson_mass = 91.2

zboson_decaywidth = 2.5

weinberg_angle = 0.223

electron_charge = -1

upquark_charge = 2/3

downquark_charge = -1/3

upquark_isospin = 1/2

downquark_isospin = -1/2

number_of_quark_flavours = 5

number_of_qcd_colours = 3

conversion_factor = 3.89379656*10**8

kappa = 1/(4*weinberg_angle*(1-weinberg_angle))

electron_vectorcoupling = downquark_isospin - 2* electron_charge*weinberg_angle

upquark_vectorcoupling = upquark_isospin - 2* upquark_charge*weinberg_angle

downquark_vectorcoupling= downquark_isospin - 2* downquark_charge*weinberg_angle

In [2]:
uniform_distribution = 1/((zboson_mass + 3*zboson_decaywidth)**2-(zboson_mass - 3*zboson_decaywidth)**2)

def chi_1(s): 

    return kappa* (s*(s- zboson_mass**2))/((s- zboson_mass**2)**2 + zboson_decaywidth**2 * zboson_mass**2)

def chi_2(s):

    return kappa**2 * (s**2)/((s- zboson_mass**2)**2 + zboson_decaywidth**2 * zboson_mass**2)
    
#defining the matrix element e^+ e^- --> qq

def matrix_element(s,cos_theta,quark_flavour):

    if (quark_flavour == 2 or quark_flavour ==5): 
        return (4*pi*qed_coupling)**2 * number_of_qcd_colours* ((1+ cos_theta**2)*(electron_charge**2*upquark_charge**2 + 2* electron_charge * upquark_charge* electron_vectorcoupling * upquark_vectorcoupling * chi_1(s) + (downquark_isospin**2+ electron_vectorcoupling**2)*(upquark_isospin**2 + upquark_vectorcoupling**2)*chi_2(s))+ cos_theta* (4* electron_charge*upquark_charge*downquark_isospin*upquark_isospin*chi_1(s)+ 8* downquark_isospin*electron_vectorcoupling* upquark_isospin* upquark_vectorcoupling*chi_2(s)))
    
    else:  
        return (4*pi*qed_coupling)**2 * number_of_qcd_colours* ((1+ cos_theta**2)*(electron_charge**2*downquark_charge**2 + 2* electron_charge * downquark_charge* electron_vectorcoupling * downquark_vectorcoupling * chi_1(s) + (downquark_isospin**2+ electron_vectorcoupling**2)*(downquark_isospin**2 + downquark_vectorcoupling**2)*chi_2(s))+ cos_theta* (4* electron_charge*downquark_charge*downquark_isospin*downquark_isospin*chi_1(s)+ 8* downquark_isospin*electron_vectorcoupling* downquark_isospin* downquark_vectorcoupling*chi_2(s)))

def differential_cross_section(s,cos_theta):

    summed_matrix_elements = 0
    
    for i in range(number_of_quark_flavours):
    
        quark_flavour = i+1
        
        summed_matrix_elements = summed_matrix_elements + matrix_element(s,cos_theta,quark_flavour)
    
    return conversion_factor *uniform_distribution* 1/(8*pi) *1/(4*pi) * 1/(2*s) *summed_matrix_elements

In [3]:
#monte carlo integrator with histogram
def mc_integrator(function, low_limits, up_limits, number_of_samples):
    
    # Initialize histogram
    nbins = 100
    s_min = low_limits[0]
    s_max = up_limits[0]
    histogram = Histo1D(nbins, s_min, s_max)
    function_value = []
    dimension = len(low_limits)
    
    integration_volume = np.prod(up_limits- low_limits)
    
    for _ in range(number_of_samples):
        
        sample = rng.uniform(low_limits,up_limits, size=(dimension))
        
        f= function(sample[0],sample[1])
        
        function_value.append(f*integration_volume/number_of_samples)
    
        histogram.fill(sample[0],f/number_of_samples*integration_volume)
        
    error =  np.std(function_value) * np.sqrt(number_of_samples)
    
    integral =  np.sum(function_value)
    
    return integral,histogram,error

In [4]:
#monte carlo estimate for flat beam spectrum 
s_lowlimit = (zboson_mass - 3*zboson_decaywidth)**2
s_uplimit  = (zboson_mass + 3*zboson_decaywidth)**2

low_limits = np.array([s_lowlimit,-1,0])
up_limits = np.array([s_uplimit,1,2*pi])

integration_value_flat_beam = mc_integrator(differential_cross_section,low_limits,up_limits,50000)
print(integration_value_flat_beam[0],integration_value_flat_beam[2]) #compatible with 9880(50) pb

10001.963494829279 55.50856135343119


In [5]:
#monte carlo error estimate for different number of samples
s_lowlimit = (zboson_mass - 3*zboson_decaywidth)**2
s_uplimit  = (zboson_mass + 3*zboson_decaywidth)**2

low_limits = np.array([s_lowlimit,-1,0])
up_limits = np.array([s_uplimit,1,2*pi])


number_of_samples = [10,25,50,100,200,500,1000,2000,5000,7500,10000,20000,50000,100000]
integration_values_mcerror  = [mc_integrator(differential_cross_section,low_limits,up_limits,i) for i in number_of_samples]
integration_values_mcerror = np.array(integration_values_mcerror)

with open('../Data_and_Plots/data1.1dPart1.txt', 'wb') as f:
    header = 'Number of Samples  MCerror\n'
    header_ascii = header.encode('ascii')
    f.write(header_ascii)
    for i in range(len(number_of_samples)):
        line = str(number_of_samples[i])+ ' ' + str(integration_values_mcerror[:,2][i]) + '\n'
        line_ascii = line.encode('ascii')
        f.write(line_ascii)

In [6]:
s_lowlimit = (zboson_mass - 3*zboson_decaywidth)**2
s_uplimit  = (zboson_mass + 3*zboson_decaywidth)**2

low_limits = np.array([s_lowlimit,-1,0])
up_limits = np.array([s_uplimit,1,2*pi])

integration_values_1c = mc_integrator(differential_cross_section,low_limits,up_limits,20000)

histo = integration_values_1c[1]

with open('../Data_and_Plots/data1.1dPart2.txt', 'wb') as f:
    ascii_histo = str(histo).encode('ascii')
    f.write(ascii_histo)

In [7]:
#monte carlo integrator for fixed s values
def mc_integrator_fixed_s(function,s, low_limits, up_limits, number_of_samples):
    
    dimension = len(low_limits)
    
    integration_volume = np.prod(up_limits- low_limits)
    
    samples = rng.uniform(low_limits, up_limits, size=(number_of_samples, dimension))
    
    function_values = function(s,samples[:,0])
    
    integral = integration_volume * np.mean(function_values)
    
    error = integration_volume * np.std(function_values) / np.sqrt(number_of_samples)
    
    return integral

In [8]:
def differential_cross_section_fixed_s(s,cos_theta):

    summed_matrix_elements = 0
    
    for i in range(number_of_quark_flavours):
    
        quark_flavour = i+1
        
        summed_matrix_elements = summed_matrix_elements + matrix_element(s,cos_theta,quark_flavour)
    
    return conversion_factor * 1/(8*pi) *1/(4*pi) * 1/(2*s) *summed_matrix_elements

In [9]:
# mc estimate for the range of s values used in the flat beam spectrum

s_lowlimit = (zboson_mass - 3*zboson_decaywidth)**2
s_uplimit  = (zboson_mass + 3*zboson_decaywidth)**2

s_values = np.arange(s_lowlimit,s_uplimit,1)

low_limits = np.array([-1,0])

up_limits = np.array([1,2*pi])

number_of_samples = 20000

integral_values_1d = [mc_integrator_fixed_s(differential_cross_section_fixed_s,i,low_limits,up_limits,number_of_samples) for i in s_values]

integral_values_1d = np.array(integral_values_1d)

with open('../Data_and_Plots/data1.1dPart3.txt', 'wb') as f:
    header = 'S Values  MC Estimate\n'
    header_ascii = header.encode('ascii')
    f.write(header_ascii)
    for i in range(len(s_values)):
        line = str(s_values[i])+ ' ' + str(integral_values_1d[i]) + '\n'
        line_ascii = line.encode('ascii')
        f.write(line_ascii)